<a href="https://colab.research.google.com/github/bordeauxrouge99/test_repo/blob/master/Regex_development_for_vendors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Create regex patterns to extract vendor names from transaction descriptions using the data from the "details" tab of the Google Sheet at "https://docs.google.com/spreadsheets/d/1_Z0l6h9HO5Llt8bklSpIKMvOWHQsSuXy3i5y8nYgDMs/edit?gid=0#gid=0", specifically using the "Description" and "vendor" columns.

## Load data

### Subtask:
Load the data from the specified Google Sheet into a pandas DataFrame.


**Reasoning**:
Load the data from the Google Sheet URL into a pandas DataFrame and display the head and info.



In [136]:
#Cell 7e4aea91 - amm added, do not delete

import re
import pandas as pd
from google.colab import auth
import gspread
from google.auth import default
from pathlib import Path
import re
from typing import List, Iterable, Union, Tuple


# Authenticate and authorize access to Google Drive and Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open the spreadsheet and select the 'details' tab
google_sheet_url = "https://docs.google.com/spreadsheets/d/1_Z0l6h9HO5Llt8bklSpIKMvOWHQsSuXy3i5y8nYgDMs/edit?gid=0#gid=0"
# Extract the spreadsheet ID from the URL
spreadsheet_id = google_sheet_url.split('/')[-2]
# Open the spreadsheet by ID
sh = gc.open_by_key(spreadsheet_id)
# Select the 'details' worksheet (assuming 'details' is the name of the tab)
worksheet = sh.worksheet('details')

# Get all data from the worksheet
data = worksheet.get_all_values()

# Convert the data to a pandas DataFrame
df = pd.DataFrame(data[1:], columns=data[0])

# display(df.head())
# display(df.info())

In [137]:
# Cell to load finalized vendor patterns from a CSV file in Google Drive

# You might need to authenticate Google Drive access if you haven't already
from google.colab import drive
import os
import pandas as pd

# Mount Google Drive (will prompt for authorization the first time)
try:
    drive.mount('/content/drive')
except:
    print("Google Drive already mounted.")

# Define the path to the saved patterns file in your Google Drive
save_path = '/content/drive/My Drive/vendor_regex_patterns.csv'

# Initialize the dictionary and DataFrame for finalized patterns
finalized_vendor_patterns = {}
vendor_patterns_df = pd.DataFrame(columns=['cleaned_vendor', 'regex_pattern'])

# Check if the saved file exists and load it
if os.path.exists(save_path):
    print(f"Loading finalized patterns from: {save_path}")
    try:
        vendor_patterns_df = pd.read_csv(save_path)
        # Populate the dictionary from the loaded DataFrame
        finalized_vendor_patterns = dict(zip(vendor_patterns_df['cleaned_vendor'], vendor_patterns_df['regex_pattern']))
        print("Finalized patterns loaded successfully.")
        display(vendor_patterns_df.head())
    except Exception as e:
        print(f"Error loading patterns from CSV: {e}")
        print("Starting with empty finalized patterns.")
else:
    print(f"No saved patterns file found at {save_path}. Starting with empty finalized patterns.")

# Ensure finalized_vendor_patterns is a dictionary even if the file didn't exist or failed to load
if not isinstance(finalized_vendor_patterns, dict):
    finalized_vendor_patterns = {}

# Ensure vendor_patterns_df is a DataFrame even if the file didn't exist or failed to load
if not isinstance(vendor_patterns_df, pd.DataFrame):
    vendor_patterns_df = pd.DataFrame(columns=['cleaned_vendor', 'regex_pattern'])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading finalized patterns from: /content/drive/My Drive/vendor_regex_patterns.csv
Finalized patterns loaded successfully.


,cleaned_vendor,regex_pattern
0,aa,(?:(?:(?:\bamerican\W*airlines\w+|americanairl...
1,amazon,(?:(?i)(?!.*kindle).*(?:amazon\ \-\ |prime\ vi...
2,apple,(?:(?:\bapple\.com\b)|(?:\bitunes\b)|(?:\bAppl...
3,delta,(?:(?:\bDelta\W*Air\b|DeltaAir\w*)|(?:\bDelta\...
4,hulu,(?:(?:\bhulu\b)|(?=.*\bhlu\b)(?=.*\bhulu\b).*|...


In [138]:
# === CONFIG ===

DESC_COL = "Description"           # or "Original Description"
VENDOR_COL = "Vendor"              # your ground-truth vendor column
PRED_COL  = "New Predicted Vendor" # where we'll write predictions

# Your rules file
RULES_CSV = "/content/drive/My Drive/vendor_regex_patterns.csv"

# Columns in that CSV
RULES_VENDOR_COL  = "cleaned_vendor"
RULES_PATTERN_COL = "regex_pattern"

# (Optional) make sure Drive is mounted & the file exists with correct headers
from pathlib import Path
import pandas as pd, os

Path(RULES_CSV).parent.mkdir(parents=True, exist_ok=True)
if not os.path.exists(RULES_CSV):
    pd.DataFrame(columns=[RULES_VENDOR_COL, RULES_PATTERN_COL]).to_csv(RULES_CSV, index=False)

# Create the prediction column if missing
if PRED_COL not in df.columns:
    df[PRED_COL] = ""

print("Config set. Using:", {
    "DESC_COL": DESC_COL,
    "VENDOR_COL": VENDOR_COL,
    "PRED_COL": PRED_COL,
    "RULES_CSV": RULES_CSV,
    "RULES_VENDOR_COL": RULES_VENDOR_COL,
    "RULES_PATTERN_COL": RULES_PATTERN_COL,
})


Config set. Using: {'DESC_COL': 'Description', 'VENDOR_COL': 'Vendor', 'PRED_COL': 'New Predicted Vendor', 'RULES_CSV': '/content/drive/My Drive/vendor_regex_patterns.csv', 'RULES_VENDOR_COL': 'cleaned_vendor', 'RULES_PATTERN_COL': 'regex_pattern'}


## Prepare data

### Subtask:
Clean and prepare the vendor name and description data from the Google Sheet for building regex patterns.


**Reasoning**:
Since the previous attempt to load the data failed, I will try again using the same method as specified in the previous successful subtask's solution. I will then proceed with the data cleaning steps as outlined in the current subtask instructions.



## Build regex patterns

### Subtask:
Create regex patterns based on the cleaned vendor names and descriptions.


**Reasoning**:
Initialize an empty dictionary to store regex patterns and iterate through unique cleaned vendor names to generate patterns based on associated descriptions.



In [139]:
## STEP 1 — Define core function BASE VENDOR PATTERN

def make_vendor_pattern(
    base_terms: Iterable[Union[str, Tuple[str, ...], List[str]]],
    sep_class: str = r"\W*",                 # what can sit between words (space, punctuation, etc.)
    allow_no_space: bool = True,             # also match compact "americanair" form
    allow_suffix_on_compact: bool = True,    # compact forms accept trailing \w*
    word_boundaries: bool = True,            # wrap each OR term with \b … \b
    allow_suffix_on_spaced_tail: bool = False,  # <--- NEW: spaced terms accept trailing \w+ after the LAST word
) -> str:
    """
    Each item in base_terms can be:
      - str               -> OR term ("american air", "aa inflight")
      - tuple/list[str]   -> AND term; all tokens must appear somewhere, any order
    No inline (?i) flags; use case-insensitive matching at search time.
    """
    alts = []

    def _or_atom(term: str) -> str:
        # spaced variant
        esc = re.escape(term).replace(r"\ ", sep_class)
        if word_boundaries:
            spaced = rf"\b{esc}\b"
        else:
            spaced = esc

        # NEW: allow trailing suffix on spaced multi-word terms like "AMERICAN AIRxxxxxxxx"
        if allow_suffix_on_spaced_tail and " " in term:
            if word_boundaries:
                # replace the trailing \b with \w+
                spaced = spaced[:-2] + r"\w+"
            else:
                spaced = spaced + r"\w+"

        parts = [spaced]

        # compact (no-space) variant
        if allow_no_space and " " in term:
            compact = re.escape(term.replace(" ", ""))
            if allow_suffix_on_compact:
                parts.append(rf"{compact}\w*")
            else:
                parts.append(rf"{compact}\b" if word_boundaries else compact)

        return "(?:" + "|".join(parts) + ")"

    for raw in base_terms:
        # AND semantics: require all tokens to appear (any order) via lookaheads
        if isinstance(raw, (list, tuple)):
            toks = [str(t).strip() for t in raw if str(t).strip()]
            if not toks:
                continue
            lookaheads = []
            for t in toks:
                esc = re.escape(t).replace(r"\ ", sep_class)
                piece = rf"\b{esc}\b" if word_boundaries else esc
                # allow compact variant if token has spaces
                if allow_no_space and " " in t:
                    compact = re.escape(t.replace(" ", ""))
                    comp_piece = rf"{compact}\w*" if allow_suffix_on_compact else (rf"{compact}\b" if word_boundaries else compact)
                    piece = f"(?:{piece}|{comp_piece})"
                lookaheads.append(rf"(?=.*{piece})")
            alts.append("".join(lookaheads) + r".*")
            continue

        # Regular OR term
        if isinstance(raw, str):
            term = raw.strip()
            if term:
                alts.append(_or_atom(term))

    return rf"(?:{'|'.join(alts)})" if alts else r"(?!)"


In [140]:
#STEP 1A - a few complext vendor specific rules

# === Build pattern with optional vendor-specific extras ===

# (Optional) Apple dash prefix bits if you want to reuse later
APPLE_WS   = r"[ \t\u00A0\u202F\u2007]*"
APPLE_DASH = r"[-\u2010\u2011\u2012\u2013\u2014\u2015\u2212:]"

def _vendor_key(v: str) -> str:
    return str(v).strip().casefold()

def build_vendor_pattern(
    vendor: str,
    base_terms,
    *,
    allow_suffix_on_spaced_tail: bool = False,
    # if you ever need to override builder defaults, pass them through via **kwargs
    **kwargs
) -> str:
    """
    Wraps make_vendor_pattern(); for certain vendors, OR-in a few extra needles.
    Keep this small and declarative.
    """
    # 1) Build the generic pattern from your terms
    base_pat = make_vendor_pattern(
        base_terms,
        allow_suffix_on_spaced_tail=allow_suffix_on_spaced_tail,
        **kwargs
    )

    v = _vendor_key(vendor)
    extras = []
    guarded = None # Initialize guarded to None

    # ---- AA / American Airlines special cases ----
    if v in {"american airlines", "american air", "aa"}:
        # a) "AMERICAN" immediately followed by masked digits/X (NOT "American Express")
        extras.append(r"\bamerican(?!\W*express\b)(?:\W*[xX\d]{4,})")

        # b) "AMERICAN AIR" with masked digits/X after AIR (restrict \w+ to avoid 'airbnb')
        extras.append(r"\bamerican\W*air(?:[xX\d]{2,})")

    # ---- NYTimes ----
    if v in {"nytimes", "new york times", "nyt"}:
        # Guard the whole line: not Apple.com/Bill and not "Times Sq", then apply base terms
        guarded = rf"^(?!.*\bapple(?:\.com/bill)?\b)(?!.*\btimes\s*sq\b).*(?:{base_pat})"
        return guarded  # for NYT, use ONLY the guarded pattern (do not OR with base_pat)

    # ---- COURSERA ----
    if v in {"coursera"}:
    # Masked/compact statement style like COURSRA<alphanum...>
      extras.append(r"\bcoursra\w+")
    # (Optional) if you also see forms that start with 'coursera' and then more word chars:
      extras.append(r"\bcoursera\w*")

    # ---- ABT: prefer tickets/box-office, exclude the bar ----
    if v in {"abt"}:
    # Build the generic base pattern first
    # (this code is already doing base_pat = make_vendor_pattern(...))
    # Guard whole line: if it contains "bar" near the met-opera phrase, skip.
    # Simple/global guard usually works well:
      guarded = rf"^(?!.*\bBARNEW\b).*(?:{base_pat})"
    # return guarded # Remove this return statement

    # ---- Apple (optional example you can enable later) ----
    # if v == "apple":
    #     # Anchored "Apple — " prefix, excluding Apple Pay
    #     extras.append(rf"^\s*apple(?!\s*pay)\b(?:{APPLE_WS}{APPLE_DASH}{APPLE_WS})?")

    # (Add other vendor-specific extras here as you discover patterns)
    # e.g., if v == "delta": extras.append(r"\bdelta(?:\W*[xX\d]{4,})")

    if guarded is not None: # Check if guarded was set for a specific vendor
        return guarded
    elif extras:
        extra_or = "|".join(extras)
        return rf"(?:{base_pat}|{extra_or})"
    return base_pat

In [141]:
#this is to create the function definition
#do not delete

def analyze_vendor_descriptions(vendor_name, df):
    """
    Retrieves and displays distinct descriptions for a given vendor and suggests a regex pattern.

    Args:
        vendor_name (str): The name of the vendor to analyze.
        df (pd.DataFrame): The DataFrame containing the transaction data.
    """
    # Get distinct descriptions for the vendor
    vendor_descriptions = df[df['Vendor'] == vendor_name]['Description'].unique().tolist()

    if not vendor_descriptions:
        print(f"No descriptions found for vendor: '{vendor_name}'")
        return

    print(f"Distinct Descriptions for '{vendor_name}':")
    for desc in vendor_descriptions:
        print(f"- {desc}")

    # Suggest a regex pattern based on common terms in descriptions
    # This is a basic suggestion and can be refined.
    # We can analyze the descriptions to find common words or patterns.

    # A simple approach: find words that appear frequently in the descriptions for this vendor
    # and less frequently in descriptions for other vendors.
    all_descriptions = df['Description'].tolist()
    other_vendors_descriptions = df[df['Vendor'] != vendor_name]['Description'].tolist()

    vendor_words = ' '.join(vendor_descriptions).split()
    other_words = ' '.join(other_vendors_descriptions).split()

    # Count word frequencies
    vendor_word_counts = pd.Series(vendor_words).value_counts()
    other_word_counts = pd.Series(other_words).value_counts()

    # Identify words that are relatively unique to this vendor
    # (appearing frequently for the vendor and infrequently elsewhere)
    suggested_terms = []
    for word, count in vendor_word_counts.items():
        if count > 1 and other_word_counts.get(word, 0) < count / 2: # Appearing at least twice and at least twice as often as in other descriptions
             # Clean the word for regex (remove punctuation, make lowercase)
            cleaned_word = re.sub(r'[^\w]+', '', word).lower()
            if cleaned_word and len(cleaned_word) > 2: # Exclude empty or very short cleaned words
                 suggested_terms.append(re.escape(cleaned_word))

    # Create a regex pattern from the suggested terms
    if suggested_terms:
        # Join with OR and add case-insensitivity and word boundaries
        pattern = r'(?i)\b(?:' + '|'.join(suggested_terms) + r')\b'
    else:
        # Fallback to a simple pattern based on the vendor name
        pattern = r'(?i).*\b' + re.escape(vendor_name).replace(r'\ ', r'\s+') + r'\b.*'


    print(f"\nSuggested Regex Pattern for '{vendor_name}':")
    print(pattern)

# Example of how to use the function:
# analyze_vendor_descriptions('kindle', df) # Replace 'kindle' with the vendor you want to analyze

In [142]:
# do not delete - QA helper to debug vendors

# --- Debug helper: build pattern, test it on a text column, and show samples ---
import re, pandas as pd
from typing import Iterable, Union, Tuple, List, Dict, Any, Optional

def debug_vendor(
    base_terms: Iterable[Union[str, Tuple[str, ...], List[str], Dict[str, Any]]],
    texts: Union[pd.Series, List[str], Iterable[str]],
    *,
    pat: Optional[str] = None,
    sample_k: int = 8,
    random_state: int = 42,
    build_kwargs: Optional[Dict[str, Any]] = None,
):
    """
    Build a regex from base_terms (unless pat is given), test against `texts`,
    and return a summary with sample matches / non-matches.

    Parameters
    ----------
    base_terms : list/tuple/dict terms (same schema your builder accepts)
    texts      : pandas Series or any iterable of strings (e.g., df[DESC_COL])
    pat        : if provided, use this regex instead of building from base_terms
    sample_k   : number of examples to show from each of matches/unmatched
    build_kwargs : forwarded to make_vendor_pattern (e.g., {'allow_suffix_on_spaced_tail': True})

    Returns
    -------
    dict with keys:
      - pattern
      - matched (int)
      - total (int)
      - sample_matches (list[str])
      - sample_unmatched (list[str])
      - mask (pd.Series[bool])  # same index as input Series if a Series was passed
    """
    # Normalize to a Series and keep index if possible
    ser = pd.Series(texts, copy=False)
    ser = ser.astype(str)

    # Build or use provided pattern
    if pat is None:
        build_kwargs = build_kwargs or {}
        pattern = make_vendor_pattern(base_terms, **build_kwargs)
    else:
        pattern = pat

    # Evaluate (case-insensitive, robust to NaN)
    mask = ser.str.contains(pattern, regex=True, case=False, na=False)

    # Sampling
    matched = int(mask.sum())
    total = int(mask.size)
    unmatched = total - matched

    if matched:
        sample_matches = ser[mask].sample(n=min(sample_k, matched), random_state=random_state).tolist()
    else:
        sample_matches = []

    if unmatched:
        sample_unmatched = ser[~mask].sample(n=min(sample_k, unmatched), random_state=random_state).tolist()
    else:
        sample_unmatched = []

    return {
        "pattern": pattern,
        "matched": matched,
        "total": total,
        "sample_matches": sample_matches,
        "sample_unmatched": sample_unmatched,
        "mask": mask if isinstance(texts, pd.Series) else pd.Series(mask),
    }

def print_debug_vendor(res):
    """Pretty-print the dict returned by debug_vendor()."""
    print("Pattern:", res["pattern"])
    print(f"Matched: {res['matched']} / {res['total']}")
    print("Sample matches:")
    for m in res["sample_matches"]:
        print("  •", m)
    print("\nSample non-matches (to inspire variants):")
    for u in res["sample_unmatched"]:
        print("  ×", u[:120])


### Mini sanity test (you can run this and then delete it)

1.   List item
2.   List item



In [143]:
# Test the builder on a few synthetic strings (optional)
pat = make_vendor_pattern(["nyc taxi","nyctaxi","taxi into work"])
tests = [
    ("apple", True),
    ("itunes", True),
    ("Taxi into Work – Monthly", True),
    ("random text", False),
]

ok = all(bool(re.search(pat, s, re.I)) == e for s, e in tests)
print("Pattern:", pat)
print("Sanity OK:", ok)

Pattern: (?:(?:\bnyc\W*taxi\b|nyctaxi\w*)|(?:\bnyctaxi\b)|(?:\btaxi\W*into\W*work\b|taxiintowork\w*))
Sanity OK: False


## STEP 2 — Review top vendors to prioritize (focus your time)

RESTART THE PROCESS HERE
EITHER PICK A VENDOR
OR
START AT THE TOP


Do you want to take the vendor with the most instances or a specific one?

In [144]:
# BRIDGE: merge CSV rules into finalized_vendor_patterns before the KEEP block


# Ensure the dict exists
if "finalized_vendor_patterns" not in globals() or not isinstance(finalized_vendor_patterns, dict):
    finalized_vendor_patterns = {}

# Load vendors from local CSV (RULES_CSV must be set earlier)
rules_path = Path(RULES_CSV)
if rules_path.exists():
    rules_df = pd.read_csv(rules_path)
    # normalize columns
    rules_df.columns = [c.strip().lower().replace(" ", "_") for c in rules_df.columns]
    vend_col = next((c for c in ["canonical_vendor", "vendor", "vendor_key"] if c in rules_df.columns), None)
    pat_col  = "pattern" if "pattern" in rules_df.columns else None

    if vend_col:
        for _, r in rules_df.iterrows():
            v = str(r[vend_col]).strip()
            if not v:
                continue
            # keep any existing pattern; otherwise stash from CSV (or a placeholder)
            if v not in finalized_vendor_patterns:
                finalized_vendor_patterns[v] = (str(r[pat_col]) if pat_col and pd.notna(r.get(pat_col)) else "CSV_NO_PATTERN")

    print("Loaded vendors from CSV:", sorted(list({k for k in finalized_vendor_patterns.keys()}))[:12],
          "..." if len(finalized_vendor_patterns) > 12 else "")
else:
    print(f"No rules CSV at {rules_path} — skipping CSV merge.")


Loaded vendors from CSV: ['7th_st_burger', 'aa', 'abt', 'allianz', 'amazon', 'amc_theatres', 'apple', 'balthazar', 'bite', 'blaze', 'blue_apron', 'cibo'] ...


In [145]:
##KEEP
#cell 462d8728 - audrey added, do not delete

# Group by 'Vendor' and count the number of unique descriptions for each vendor
unique_description_counts = df.groupby('Vendor')['Description'].nunique()

# Exclude the vendor '0'
unique_description_counts = unique_description_counts[unique_description_counts.index != '0']

# Exclude vendors for which we have finalized patterns
vendors_to_exclude = finalized_vendor_patterns.keys()
unique_description_counts = unique_description_counts.drop(vendors_to_exclude, errors='ignore')

# Sort the results by the count of distinct descriptions in descending order
unique_description_counts = unique_description_counts.sort_values(ascending=False)

# Display the top 20 vendors and their distinct description counts
print("Top 20 Vendors and Distinct Description Counts (Excluding '0' and vendors with finalized patterns), sorted by count descending:")
print(unique_description_counts.head(20))

Top 20 Vendors and Distinct Description Counts (Excluding '0' and vendors with finalized patterns), sorted by count descending:
Vendor
credit_card           13
uber_eats              5
title_9                5
playhouse_sq           5
gruppo                 5
east_yoga              5
billie                 5
ticketmaster           5
mt_sinai               5
motel_morris           5
olympic_center         5
maison_kayser          5
arc_teryx              5
la_bergamote           5
caf_cluny              5
arte                   5
hotel_arenal_manoa     5
fedex                  5
hot_topic              5
plan_fee               5
Name: Description, dtype: int64


INPUT VENDOR NAME HERE

In [146]:
# Set the current working vendor

TARGET_VENDOR = 'stitch_fix' ##INPUT

print(f"Current working vendor set to: {TARGET_VENDOR}")

Current working vendor set to: stitch_fix


In [147]:
import pandas as pd, re
from pathlib import Path

# ------- Inputs expected to exist -------
# df, TARGET_VENDOR, DESC_COL, VENDOR_COL
# RULES_CSV, RULES_VENDOR_COL="cleaned_vendor", RULES_PATTERN_COL="regex pattern"

vn = TARGET_VENDOR.strip()
vn_cf = vn.casefold()

# Rows labeled with this vendor
vend_norm = df[VENDOR_COL].astype(str).str.strip().str.casefold()
vendor_df = df.loc[vend_norm.eq(vn_cf), [DESC_COL, VENDOR_COL]].copy()

# Missing vs non-missing descriptions
missing_desc = vendor_df[DESC_COL].isna() | vendor_df[DESC_COL].astype(str).str.strip().eq("")
with_desc_df = vendor_df.loc[~missing_desc].copy()

# Counts (rows, not distinct)
rows_total         = int(len(vendor_df))
rows_with_desc     = int(len(with_desc_df))
rows_missing_desc  = int(missing_desc.sum())

print(f"Vendor = '{vn}'")
print(f"  • total rows (label-based): {rows_total}")
print(f"  • rows with a non-empty Description: {rows_with_desc}")
print(f"  • rows missing/blank Description:   {rows_missing_desc}")

# Distinct descriptions to “solve” with regex (exact-text distincts)
distinct_desc = sorted(with_desc_df[DESC_COL].astype(str).str.strip().unique())
print(f"\nDistinct descriptions to cover (exact): {len(distinct_desc)}")

# Optional: normalized distincts (lower + collapse spaces) if you prefer that notion of “distinct”
distinct_norm = sorted(
    pd.Series(with_desc_df[DESC_COL].astype(str).str.strip()
              .str.lower().str.replace(r"\s+", " ", regex=True)).unique()
)
print(f"Distinct descriptions to cover (normalized): {len(distinct_norm)}")

# Random sample of up to 10 distinct descriptions (exact version)
k = min(10, len(distinct_desc))
if k > 0:
    sample = pd.Series(distinct_desc).sample(n=k, replace=False, random_state=42).sort_values()
    print(f"\nRandom sample of {k} distinct descriptions:")
    for s in sample.tolist():
        print(" -", s)
else:
    print("\nNo descriptions to sample.")

# (Optional) Save all distinct descriptions to a CSV to work on the regex
# pd.DataFrame({DESC_COL: distinct_desc}).to_csv("/content/target_vendor_descriptions.csv", index=False)

# ------- Optional: check coverage vs your existing CSV rule (if present) -------

if 'with_desc_df' not in locals():
    vend_norm = df[VENDOR_COL].astype(str).str.strip().str.casefold()
    vendor_df = df.loc[vend_norm.eq(vn_cf), [DESC_COL, VENDOR_COL]].copy()
    missing_desc = vendor_df[DESC_COL].isna() | vendor_df[DESC_COL].astype(str).str.strip().eq("")
    with_desc_df = vendor_df.loc[~missing_desc].copy()
rows_with_desc = int(len(with_desc_df))

# Load rules CSV
path = Path(RULES_CSV)
if not path.exists():
    print(f"[Coverage check skipped] Rules CSV not found at {path}")
else:
    rules = pd.read_csv(path)

    # --- Normalize column names and resolve your configured schema ---
    import re as _re
    def _norm_col(s): return _re.sub(r'[\s_]+', '', str(s).strip().lower())

    # Map normalized -> original column name
    col_map = {_norm_col(c): c for c in rules.columns}

    # Resolve vendor/pattern columns using your CONFIG constants, with fallbacks
    vend_col = col_map.get(_norm_col(RULES_VENDOR_COL)) or col_map.get('cleanedvendor') or col_map.get('vendor') or col_map.get('vendorkey')
    pat_col  = col_map.get(_norm_col(RULES_PATTERN_COL)) or col_map.get('regexpattern')  or col_map.get('pattern') or col_map.get('regex')

    if not vend_col or not pat_col:
        print("[Coverage check skipped] Could not find expected columns in CSV.")
        print("  Available CSV headers:", list(rules.columns))
        print("  Looking for vendor like:", RULES_VENDOR_COL, "| pattern like:", RULES_PATTERN_COL)
    else:
        # Find the rule row for this vendor (case-insensitive)
        rule_row = rules[rules[vend_col].astype(str).str.strip().str.casefold().eq(vn_cf)]
        if rule_row.empty:
            print(f"No existing rule found in CSV for '{vn}'.")
        else:
            patt = str(rule_row.iloc[0][pat_col]).strip()
            # strip any leading inline flags, just in case
            patt = re.sub(r'^\(\?[aiLmsux-]+\)', "", patt or "")
            ser = with_desc_df[DESC_COL].astype(str)
            # Ensure the pattern is a non-empty string before using it
            if patt:
                covered_mask = ser.str.contains(patt, regex=True, case=False, na=False)
                covered = int(covered_mask.sum())
                print(f"Existing CSV rule coverage for '{vn}': {covered}/{rows_with_desc} rows")

                # Show a few uncovered distinct descriptions (if any)
                uncovered_desc = sorted(ser[~covered_mask].str.strip().unique().tolist())
                if uncovered_desc:
                    print(f"Uncovered distinct descriptions ({len(uncovered_desc)}):")
                    for s in uncovered_desc[:10]:
                        print(" ×", s)
            else:
                print(f"Existing rule for '{vn}' has an empty or invalid pattern.")

Vendor = 'stitch_fix'
  • total rows (label-based): 7
  • rows with a non-empty Description: 7
  • rows missing/blank Description:   0

Distinct descriptions to cover (exact): 2
Distinct descriptions to cover (normalized): 2

Random sample of 2 distinct descriptions:
 - STITCH FIX SAN FRANCISCO CA
 - Stitch Fix
Existing CSV rule coverage for 'stitch_fix': 7/7 rows


In [148]:
# Analyze the descriptions for the current working vendor
analyze_vendor_descriptions(TARGET_VENDOR, df)

Distinct Descriptions for 'stitch_fix':
- Stitch Fix
- STITCH FIX SAN FRANCISCO CA

Suggested Regex Pattern for 'stitch_fix':
(?i).*\bstitch_fix\b.*


Review the patterns above. get AI to help if needed for the next cell (f53223a3)

In [149]:
# Draft base terms (start small, add variants only if needed)

base_terms = [
    "Stitch Fix",
    # "netflix.com",
    # "MABROOKLYN",
    # "Westvillenyc",
    # "ELECTRONIC PAYMENT",
    # "PAYMENT THANK YOU",
    # "Credit Card Payments",
    # "Corrective Payment"
    # ("ABT", "tickets"),
    # ("CIBO", "market"),

]
# <-- tuples: require both 'taxi' and 'astoria' anywhere



In [150]:
# MAIN DEBUG SET

# Generate PATTERN from your base_terms - >>>>>>> CHANGE THIS AS NEEDED <<<<<<<<
pat = build_vendor_pattern(
    TARGET_VENDOR,
    base_terms,
    allow_suffix_on_spaced_tail=False  # True for multi-word tails like "AMERICAN AIR"
)


res = debug_vendor(base_terms, df[DESC_COL])
print("Pattern:", res["pattern"])
print("Matched:", res["matched"])
print("Sample matches:")
for m in res["sample_matches"][:8]:
    print("  •", m)
print("\nSample non-matches (just in case):")
for u in res["sample_unmatched"][:8]:
    print("  ×", u[:120])


# Debug specifically for the TARGET_VENDOR: show covered vs. missed labeled rows
vn = TARGET_VENDOR.strip()
vn_cf = vn.casefold()


# Convenience series/masks
desc = df[DESC_COL].astype(str)
vend = df[VENDOR_COL].astype(str).str.strip().str.casefold()
mask_regex  = desc.str.contains(pat, regex=True, case=False, na=False)
mask_vendor = vend.eq(vn_cf)

# Consider only labeled rows that have a non-empty description
mask_with_desc = desc.str.strip().ne("")
mask_vendor_rows = mask_vendor & mask_with_desc

total_vendor_rows = int(mask_vendor_rows.sum())
covered_vendor_rows = int((mask_vendor_rows & mask_regex).sum())
miss_mask = mask_vendor_rows & ~mask_regex
miss_df = df.loc[miss_mask, [DESC_COL, VENDOR_COL]].copy()


# List the *missed* descriptions (these are the ones you need to account for)
if miss_df.empty:
    print("No misses — pattern covers all labeled rows for this vendor.")
else:
    print("\nDescriptions for this vendor NOT matched by the pattern:")
    for s in miss_df[DESC_COL].astype(str).tolist():
        print(" ×", s)

# (Optional) also peek at potential collisions (pattern hits other vendors)
collisions = df.loc[mask_regex & ~mask_vendor, [DESC_COL, VENDOR_COL]].head(10)
if not collisions.empty:
    print("\nSample of pattern matches on other vendors (potential collisions):")
    for _, row in collisions.iterrows():
        print(f" ! {row[DESC_COL]}  | labeled as: {row[VENDOR_COL]}")


print(f"Pattern: {pat}")
print(f"Vendor '{vn}': {covered_vendor_rows}/{total_vendor_rows} labeled rows with descriptions matched")

Pattern: (?:(?:\bStitch\W*Fix\b|StitchFix\w*))
Matched: 7
Sample matches:
  • Stitch Fix
  • Stitch Fix
  • STITCH FIX SAN FRANCISCO CA
  • STITCH FIX SAN FRANCISCO CA
  • STITCH FIX SAN FRANCISCO CA
  • STITCH FIX SAN FRANCISCO CA
  • STITCH FIX SAN FRANCISCO CA

Sample non-matches (just in case):
  × FOODKICK xxx-xxx-2785 NY
  × STARBUCKS CARD RELOAD 800-782-7282 WA
  × PAYMENT - THANK YOU
  × Amazon - water
  × APL* ITUNES.COM/BILL
  × Westville
  × BLUEAPRON* ORDER
  × ABC COCINA NEW YORK NY
No misses — pattern covers all labeled rows for this vendor.
Pattern: (?:(?:\bStitch\W*Fix\b|StitchFix\w*))
Vendor 'stitch_fix': 7/7 labeled rows with descriptions matched


#STOP DID THE REGEX PATTERN PASS ALL QA

In [151]:
# === QA GATE (merged, collision-aware) ===
# Coverage must pass; collision handling is configurable.

REQUIRE_FULL_COVERAGE = True            # require 100% of labeled rows w/ descriptions
COLLISION_SCOPE       = "unlabeled_only"  # "none" | "unlabeled_only" | "all"
MAX_COLLISIONS        = 999999           # threshold for collisions after filtering
AUTO_SAVE_IF_PASS     = False            # write CSV automatically on PASS

# OPTIONAL allow-lists to ignore known-benign matches
ALLOW_COLLISION_VENDORS  = set()         # e.g., {"nytimes","hbo max"}
ALLOW_COLLISION_PATTERNS = []            # e.g., [r"\bny ?times\b", r"\bhbo ?max\b"]

import re, pandas as pd
from pathlib import Path

vn    = TARGET_VENDOR.strip()
vn_cf = vn.casefold()

# Use your prebuilt 'pat' if it exists (e.g., special Apple prefix); otherwise build from base_terms.
try:
    pat  # keep any prior custom 'pat'
except NameError:
    pat = make_vendor_pattern(base_terms)

# Convenience series/masks
desc = df[DESC_COL].astype(str)
vend = df[VENDOR_COL].astype(str).str.strip().str.casefold()
pred = (df[PRED_COL].astype(str).str.strip().str.casefold()
        if PRED_COL in df.columns else pd.Series("", index=df.index))

mask_regex       = desc.str.contains(pat, regex=True, case=False, na=False)
mask_with_desc   = desc.str.strip().ne("")
mask_vendor      = vend.eq(vn_cf)
mask_vendor_rows = mask_vendor & mask_with_desc

# --- Coverage on labeled rows ---
total_vendor_rows   = int(mask_vendor_rows.sum())
covered_vendor_rows = int((mask_vendor_rows & mask_regex).sum())
miss_count          = total_vendor_rows - covered_vendor_rows
miss_df = df.loc[mask_vendor_rows & ~mask_regex, [DESC_COL, VENDOR_COL]].copy()

# --- Collisions with scope + allow-lists ---
if COLLISION_SCOPE == "none":
    collisions_df = df.iloc[0:0].copy()
else:
    base_collisions = df[mask_regex & ~mask_vendor].copy()
    if COLLISION_SCOPE == "unlabeled_only":
        unlabeled_mask = pred.eq("")
        base_collisions = base_collisions[unlabeled_mask.loc[base_collisions.index]]
    # allow-list by vendor
    if ALLOW_COLLISION_VENDORS:
        base_collisions = base_collisions[
            ~base_collisions[VENDOR_COL].astype(str).str.strip().str.casefold()
             .isin({v.casefold() for v in ALLOW_COLLISION_VENDORS})
        ]
    # allow-list by description regex
    if ALLOW_COLLISION_PATTERNS:
        keep = pd.Series(True, index=base_collisions.index)
        for rgx in ALLOW_COLLISION_PATTERNS:
            keep &= ~base_collisions[DESC_COL].astype(str).str.contains(rgx, regex=True, case=False, na=False)
        base_collisions = base_collisions[keep]
    collisions_df = base_collisions[[DESC_COL, VENDOR_COL]]

# --- Evaluate ---
pass_coverage   = (miss_count == 0) if REQUIRE_FULL_COVERAGE else True
pass_collisions = (len(collisions_df) <= MAX_COLLISIONS)
qa_pass = pass_coverage and pass_collisions

# --- Report ---
print("=== QA REPORT ===")
print(f"Vendor:         {vn}")
print(f"Pattern:        {pat}")
print(f"Coverage:       {covered_vendor_rows}/{total_vendor_rows} (misses: {miss_count})")
print(f"Collision mode: {COLLISION_SCOPE} | Count: {len(collisions_df)} | Threshold: {MAX_COLLISIONS}")

if miss_count:
    missed_unique = sorted(miss_df[DESC_COL].astype(str).str.strip().unique().tolist())
    print("\nMissed distinct descriptions (up to 10):")
    for s in missed_unique[:10]:
        print(" ×", s)

if len(collisions_df):
    print("\nSample collisions (up to 10 after allow-lists):")
    for _, row in collisions_df.head(10).iterrows():
        print(f" ! {row[DESC_COL]}  | labeled-as: {row[VENDOR_COL]}")
    print("\nTop 'labeled-as' vendors in collisions:")
    print(collisions_df[VENDOR_COL].astype(str).str.strip().str.lower().value_counts().head(10))

if qa_pass:
    print("\n✅ QA PASS")
    print(f"APPROVED → vendor: '{vn}', regex pattern:\n{pat}")

    # --- update local dict ---
    if "finalized_vendor_patterns" not in globals() or not isinstance(finalized_vendor_patterns, dict):
        finalized_vendor_patterns = {}
    finalized_vendor_patterns[vn] = pat
    print(f"🧠 Updated finalized_vendor_patterns['{vn}']")

    # --- optional CSV save ---
    if AUTO_SAVE_IF_PASS:
        try:
            path = Path(RULES_CSV)
            rules = pd.read_csv(path) if path.exists() else pd.DataFrame(columns=[RULES_VENDOR_COL, RULES_PATTERN_COL])
            new_row = pd.DataFrame([{RULES_VENDOR_COL: vn, RULES_PATTERN_COL: pat}])
            rules["_vn"] = rules[RULES_VENDOR_COL].astype(str).str.strip().str.casefold()
            new_row["_vn"] = new_row[RULES_VENDOR_COL].astype(str).str.strip().str.casefold()
            rules = (pd.concat([rules, new_row], ignore_index=True)
                       .drop_duplicates(subset=["_vn"], keep="last")
                       .drop(columns=["_vn"]))
            rules.to_csv(path, index=False)
            print(f"💾 Saved rule to {path}")
        except Exception as e:
            print(f"⚠️ AUTO_SAVE_IF_PASS failed: {e}")
else:
    print("\n❌ QA FAIL — refine base_terms or pattern and re-run.")


=== QA REPORT ===
Vendor:         stitch_fix
Pattern:        (?:(?:\bStitch\W*Fix\b|StitchFix\w*))
Coverage:       7/7 (misses: 0)
Collision mode: unlabeled_only | Count: 0 | Threshold: 999999

✅ QA PASS
APPROVED → vendor: 'stitch_fix', regex pattern:
(?:(?:\bStitch\W*Fix\b|StitchFix\w*))
🧠 Updated finalized_vendor_patterns['stitch_fix']


In [152]:
# === POST-QA: sync local dict & CSV, then show both ===
import pandas as pd, os, re
from pathlib import Path

# Expect these from CONFIG / prior cells:
#   TARGET_VENDOR, base_terms, pat (pattern), qa_pass
#   RULES_CSV, RULES_VENDOR_COL="cleaned_vendor", RULES_PATTERN_COL="regex pattern"

# Safety: make sure CONFIG exists
for _need in ("RULES_CSV", "RULES_VENDOR_COL", "RULES_PATTERN_COL"):
    if _need not in globals():
        raise RuntimeError("Please run the CONFIG cell first (defines RULES_CSV / RULES_VENDOR_COL / RULES_PATTERN_COL).")

vendor_to_add  = TARGET_VENDOR.strip()
pattern_to_add = (pat if "pat" in globals() else make_vendor_pattern(base_terms))

# 1) Update only if QA passed (keeps your process disciplined)
if "qa_pass" in globals() and qa_pass:
    # ---- update local dict ----
    if "finalized_vendor_patterns" not in globals() or not isinstance(finalized_vendor_patterns, dict):
        finalized_vendor_patterns = {}
    finalized_vendor_patterns[vendor_to_add] = pattern_to_add
    print(f"🧠 Local dict updated for '{vendor_to_add}'")

    # ---- update CSV (de-dup by vendor, last-write wins) ----
    path = Path(RULES_CSV)
    if not path.exists():
        pd.DataFrame(columns=[RULES_VENDOR_COL, RULES_PATTERN_COL]).to_csv(path, index=False)

    rules = pd.read_csv(path)
    rules["_vn"] = rules[RULES_VENDOR_COL].astype(str).str.strip().str.casefold()
    new = pd.DataFrame([{RULES_VENDOR_COL: vendor_to_add, RULES_PATTERN_COL: pattern_to_add}])
    new["_vn"] = new[RULES_VENDOR_COL].astype(str).str.strip().str.casefold()

    rules = (pd.concat([rules, new], ignore_index=True)
               .drop_duplicates(subset=["_vn"], keep="last")
               .drop(columns=["_vn"]))
    rules.to_csv(path, index=False)
    print(f"💾 CSV updated at: {path}")
elif "qa_pass" in globals() and not qa_pass:
    print("⚠️ QA did not pass — not updating dict/CSV. Refine pattern and re-run.")
else:
    print("ℹ️ 'qa_pass' not found — showing current state without writing.")

# 2) Show the in-memory dictionary as a DataFrame
if "finalized_vendor_patterns" in globals() and isinstance(finalized_vendor_patterns, dict) and finalized_vendor_patterns:
    mem_df = (pd.DataFrame(
        [{RULES_VENDOR_COL: k, RULES_PATTERN_COL: v} for k, v in finalized_vendor_patterns.items()]
    ).sort_values(by=[RULES_VENDOR_COL]).reset_index(drop=True))
else:
    mem_df = pd.DataFrame(columns=[RULES_VENDOR_COL, RULES_PATTERN_COL])

print("\n=== Updated DataFrame of Cleaned Vendor Names and Regex Patterns (in-memory) ===")
try:
    display(mem_df)
except Exception:
    print(mem_df.to_string(index=False))

# 3) Show the CSV-on-disk after update (or current contents)
if os.path.exists(RULES_CSV):
    csv_df = pd.read_csv(RULES_CSV)
    print("\n=== CSV on disk (after update if any) ===")
    try:
        display(csv_df)
    except Exception:
        print(csv_df.to_string(index=False))
else:
    print("\n(No CSV found at RULES_CSV — nothing to display)")


🧠 Local dict updated for 'stitch_fix'
💾 CSV updated at: /content/drive/My Drive/vendor_regex_patterns.csv

=== Updated DataFrame of Cleaned Vendor Names and Regex Patterns (in-memory) ===


,cleaned_vendor,regex_pattern
0,7th_st_burger,(?:(?:\b7th\W*Stre\b|7thStre\w*)|(?:\b7th\W*St...
1,aa,(?:(?:(?:\bamerican\W*airlines\w+|americanairl...
2,abt,^(?!.*\bBARNEW\b).*(?:(?:(?:\bMET\W*OPERA\b|ME...
3,allianz,(?:(?:\ballianz\b)|(?:\bAllianze\b))
4,amazon,(?:(?i)(?!.*kindle).*(?:amazon\ \-\ |prime\ vi...
...,...,...
82,westville,(?:(?:\bwestville\b)|(?:\bWWW\.WESTVILLENYC\.C...
83,whole_foods,(?:(?:\bWHOLEFDS\b))
84,wix,(?:(?:\bwix\.com\b))
85,ymca,(?:(?:\bymca\b)|(?:\b14th\W*St\W*Y\b|14thStY\w*))



=== CSV on disk (after update if any) ===


,cleaned_vendor,regex_pattern
0,aa,(?:(?:(?:\bamerican\W*airlines\w+|americanairl...
1,amazon,(?:(?i)(?!.*kindle).*(?:amazon\ \-\ |prime\ vi...
2,apple,(?:(?:\bapple\.com\b)|(?:\bitunes\b)|(?:\bAppl...
3,delta,(?:(?:\bDelta\W*Air\b|DeltaAir\w*)|(?:\bDelta\...
4,hulu,(?:(?:\bhulu\b)|(?=.*\bhlu\b)(?=.*\bhulu\b).*|...
...,...,...
82,7th_st_burger,(?:(?:\b7th\W*Stre\b|7thStre\w*)|(?:\b7th\W*St...
83,gramercy_gyno,(?:(?:\bGRAMERCY\W*GYNECOLOGY\b|GRAMERCYGYNECO...
84,gwynnie_bee,(?:(?:\bgwynnie\W*bee\b|gwynniebee\w*))
85,netflix,(?:(?:\bnetflix\b)|(?:\bnetflix\.com\b))


In [153]:
#STOP


In [154]:
# # --- Helper: snake_case a vendor name robustly ---
# import re, unicodedata

# def to_snake_vendor(name: str) -> str:
#     s = str(name or "").strip()
#     # normalize accents/Unicode (e.g., café -> cafe)
#     s = unicodedata.normalize("NFKD", s)
#     s = "".join(ch for ch in s if not unicodedata.combining(ch))
#     s = s.lower()
#     # collapse any non-word chunk to underscore (spaces, dashes, punctuation, emojis)
#     s = re.sub(r"[^\w]+", "_", s)
#     # trim / dedupe underscores
#     s = s.strip("_")
#     s = re.sub(r"_+", "_", s)
#     return s


In [155]:
# # === Snake-case vendors in RULES_CSV (and merge duplicates) ===
# import pandas as pd, re, unicodedata
# from pathlib import Path

# # Use your existing constants; if missing, set sensible defaults:
# RULES_CSV         = globals().get("RULES_CSV", "/content/drive/My Drive/vendor_regex_patterns.csv")
# RULES_VENDOR_COL  = globals().get("RULES_VENDOR_COL", "cleaned_vendor")
# RULES_PATTERN_COL = globals().get("RULES_PATTERN_COL", "regex pattern")

# path = Path(RULES_CSV)
# if not path.exists():
#     raise FileNotFoundError(f"Rules CSV not found: {path}")

# # --- helper: snake_case a vendor name robustly ---
# def to_snake_vendor(name: str) -> str:
#     s = str(name or "").strip()
#     s = unicodedata.normalize("NFKD", s)
#     s = "".join(ch for ch in s if not unicodedata.combining(ch))
#     s = s.lower()
#     s = re.sub(r"[^\w]+", "_", s)   # spaces/dashes/punct -> _
#     s = s.strip("_")
#     s = re.sub(r"_+", "_", s)       # collapse multiple _
#     return s

# # --- helper: OR-merge regex patterns cleanly ---
# def _inner(p: str) -> str:
#     p = (p or "").strip()
#     if p.startswith('(?:') and p.endswith(')'):
#         return p[3:-1]              # strip outer non-capturing group
#     return p

# def or_merge_patterns(patterns):
#     parts = [_inner(p) for p in patterns if str(p or "").strip()]
#     if not parts:
#         return ""
#     seen, uniq = set(), []
#     for part in parts:
#         if part not in seen:
#             seen.add(part)
#             uniq.append(part)
#     return "(?:" + "|".join(uniq) + ")"

# # --- load, normalize, merge ---
# rules = pd.read_csv(path)
# if RULES_VENDOR_COL not in rules.columns or RULES_PATTERN_COL not in rules.columns:
#     raise ValueError(f"CSV must contain columns: {RULES_VENDOR_COL!r}, {RULES_PATTERN_COL!r}")

# before_unique = rules[RULES_VENDOR_COL].nunique(dropna=False)

# # snake_case vendor names
# rules[RULES_VENDOR_COL] = rules[RULES_VENDOR_COL].map(to_snake_vendor)

# # group and OR-merge patterns for duplicate vendors
# merged = (rules
#           .groupby(RULES_VENDOR_COL, as_index=False)[RULES_PATTERN_COL]
#           .apply(lambda s: or_merge_patterns(s.tolist())))

# after_unique = merged[RULES_VENDOR_COL].nunique(dropna=False)

# # one-time backup (if not already made)
# bak = path.with_suffix(path.suffix + ".bak")
# if not bak.exists():
#     merged.to_csv(bak, index=False)
#     print(f"📦 Backup written: {bak}")

# # write back
# merged.to_csv(path, index=False)
# print(f"✅ Snake-cased + merged rules saved: {path}")
# print(f"   Vendors before: {before_unique}  |  after: {after_unique}")

# # (Optional) peek at a few rows
# try:
#     display(merged.head(10))
# except Exception:
#     print(merged.head(10).to_string(index=False))


In [156]:
## to resolve at the end
# - payments
# credit_cards - need to redo - didn't have full list and maybe changed them - maybe want to take more data into it to decide to i want the credit card name here rather than credit_card
# amex > do i figure out what/who the refund is for or just leave it as amex i..e, rideshare credit > is really more for uber, but it is also a reward